In [1]:
import pandas as pd
import sqlite3

#connect to database
conn = sqlite3.connect("OAdb.db")

cursor = conn.cursor()

In [2]:
# Create tables
# Only run this cell once to connect it to the same database
Brands = pd.read_csv('brands.csv').to_sql('Brands', conn)
Receipt_items = pd.read_csv('receipt_items.csv').to_sql('Receipt_items', conn)
Receipts = pd.read_csv('receipts.csv').to_sql('Receipts', conn)
Users = pd.read_csv('users.csv').to_sql('Users', conn)

ValueError: Table 'Brands' already exists.

In [3]:
# Question 1
q1string = """
SELECT B.NAME
FROM Receipt_items AS I
INNER JOIN Receipts AS R ON I.REWARDS_RECEIPT_ID = R.ID 
INNER JOIN Brands AS B ON B.BARCODE = I.BARCODE
WHERE R.PURCHASE_DATE LIKE '%-06-%' 
GROUP BY B.NAME
ORDER BY SUM(I.TOTAL_FINAL_PRICE) DESC
LIMIT 1
"""
q1 = pd.read_sql(q1string, conn)
q1

,NAME
0,Brooks®


In [5]:
# Question 2
q2string = """
SELECT U.ID, R.TOTAL_SPENT, R.PURCHASE_DATE
FROM Users AS U
INNER JOIN Receipts AS R on R.USER_ID = U.ID
WHERE R.PURCHASE_DATE LIKE '%-08-%' AND R.TOTAL_SPENT = (SELECT MAX(R.TOTAL_SPENT)
                                                        FROM Receipts AS R)
ORDER BY R.TOTAL_SPENT DESC
"""
q2 = pd.read_sql(q2string, conn)
q2

,ID,TOTAL_SPENT,PURCHASE_DATE
0,609ab37f7a2e8f2f95ae968f,93015.18,2021-08-17T00:00:00Z


In [14]:
# Question 3
q3string = """
SELECT U.ID, R.PURCHASE_DATE, R.TOTAL_SPENT
FROM Users AS U
INNER JOIN Receipts AS R on R.USER_ID = U.ID
INNER JOIN Receipt_items AS I on I.REWARDS_RECEIPT_ID = R.ID
WHERE I.TOTAL_FINAL_PRICE = (SELECT MAX(I.TOTAL_FINAL_PRICE)
                            FROM Receipt_items AS I)
GROUP BY U.ID
"""
q3 = pd.read_sql(q3string, conn)
q3

,ID,PURCHASE_DATE,TOTAL_SPENT
0,617376b8a9619d488190e0b6,2022-07-06T00:00:00Z,31018.97


In [11]:
# Question 4
q4string = """
SELECT I.DESCRIPTION
FROM Receipt_items AS I
WHERE I.TOTAL_FINAL_PRICE = (SELECT MAX(I.TOTAL_FINAL_PRICE)
                            FROM Receipt_items AS I)
GROUP BY I.Description
"""
q4 = pd.read_sql(q4string, conn)
q4

,DESCRIPTION
0,Starbucks Iced Coffee Premium Coffee Beverage ...


In [12]:
# Question 5
q5string = """
SELECT COUNT(U.ID), SUBSTRING(R.DATE_SCANNED, 6, 2) AS MONTH
FROM Users AS U
INNER JOIN Receipts AS R ON R.USER_ID = U.ID
GROUP BY MONTH
"""
q5 = pd.read_sql(q5string, conn)
q5

,COUNT(U.ID),MONTH
0,4222,01
1,3830,02
2,4767,03
3,4882,04
4,5627,05
5,5405,06
6,6058,07
7,6191,08
8,6355,09
9,7305,10


In [ ]:
cursor.close()
conn.close()
